In [1]:
import h5py
import numpy as np
import os
import random
from os.path import join
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten, Activation, TimeDistributed, LSTM
from keras.utils import to_categorical
from keras_utils import set_keras_session


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
set_keras_session()
dataset = 'UCF11'
nb_classes = int(dataset[3:])

In [3]:
def train_generator(dataset_dir, batch_size):
    
    all_files = os.listdir(dataset_dir)
    
    while True:
        images = []
        labels = []

        for _ in range(batch_size):
            
            random_filename = random.choice(all_files)
            
            with h5py.File(join(dataset_dir, random_filename), 'r') as hf:
                frames = hf['X'][:]
                # one label
                fr_labels = hf['Y'][:]
                #random_idx = np.random.randint(frames.shape[0])
                
                images.append(frames)
                labels.append(fr_labels)
                
        sequences = np.array(images)
        labels = to_categorical(np.array(labels), nb_classes)
        
        yield sequences, labels
        
        

def valid_generator(dataset_dir, batch_size):
    
    all_files = os.listdir(dataset_dir)
    
    while True:
        data_frames = []
        labels = []
        idx = 0
        for i in range(batch_size):
            if idx + i < len(all_files):
                filename = all_files[i + idx]

                with h5py.File(join(dataset_dir, filename), 'r') as hf:
                    frames = hf['X'][:]
                    single_label = hf['Y'][:][0]

                    data_frames.append(frames)
                    labels.append(single_label)
                
                if i == batch_size - 1:
                    idx += 4
            else:
                idx = 0
        
        sequences = np.array(data_frames)
        labels = to_categorical(np.array(labels), nb_classes)
        yield sequences, labels

In [4]:
separate_dataset_dir = join('datasets', dataset, 'separate_frames_50_h_120_w_160')
train_dir = join(separate_dataset_dir, 'train')
valid_dir = join(separate_dataset_dir, 'valid')

train_samples_count = len(os.listdir(train_dir))
valid_samples_count = len(os.listdir(valid_dir))

with h5py.File(join(train_dir, os.listdir(train_dir)[0])) as hf:
    image_shape = hf['X'][:].shape[:]
    print('Image input shape is', image_shape)

Image input shape is (50, 120, 160, 3)


In [10]:
def build_model():
    model = Sequential()
    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',
                     input_shape=image_shape), input_shape=image_shape))
    model.add(Activation('relu'))
    model.add(TimeDistributed(Conv2D(32, (3, 3))))
    model.add(Activation('relu'))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(Dropout(0.25))

    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same')))
    model.add(Activation('relu'))
    model.add(TimeDistributed(Conv2D(64, (3, 3))))
    model.add(Activation('relu'))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(Dropout(0.25))

    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same')))
    model.add(Activation('relu'))
    model.add(TimeDistributed(Conv2D(64, (3, 3))))
    model.add(Activation('relu'))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(Dropout(0.25))

    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same')))
    model.add(Activation('relu'))
    model.add(TimeDistributed(Conv2D(64, (3, 3))))
    model.add(Activation('relu'))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(Dropout(0.25))

    model.add(TimeDistributed(Flatten()))
    #model.add(Dense(512))
    #model.add(Activation('relu'))
    #model.add(Dropout(0.5))
    
    return model

In [12]:
batch_size = 4

from keras import metrics
from functools import partial

top_3_k_categorical_accuracy = partial(metrics.top_k_categorical_accuracy, k=3)
top_3_k_categorical_accuracy.__name__ = 'top_3'

cnn = build_model()
model = Sequential()
model.add(cnn)
model.add(LSTM(128))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
print(model.summary())
model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy', top_3_k_categorical_accuracy])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_5 (Sequential)    (None, 50, 2560)          213280    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               1376768   
_________________________________________________________________
dense_1 (Dense)              (None, 11)                1419      
_________________________________________________________________
activation_17 (Activation)   (None, 11)                0         
Total params: 1,591,467
Trainable params: 1,591,467
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
history = model.fit_generator(train_generator(train_dir, batch_size),
                    steps_per_epoch=train_samples_count // batch_size, 
                    validation_data=valid_generator(valid_dir, batch_size),
                    validation_steps=valid_samples_count // batch_size,
                    epochs=20)

Epoch 1/20
236/236 [==============================] - 75s 319ms/step - loss: 2.4513 - acc: 0.0922 - top_3: 0.2511 - val_loss: 2.5445 - val_acc: 0.0000e+00 - val_top_3: 0.0000e+00
Epoch 2/20
236/236 [==============================] - 72s 305ms/step - loss: 2.4059 - acc: 0.1229 - top_3: 0.3061 - val_loss: 2.4035 - val_acc: 0.0000e+00 - val_top_3: 0.0000e+00
Epoch 3/20
183/236 [======================>.......] - ETA: 14s - loss: 2.4056 - acc: 0.1093 - top_3: 0.3169

KeyboardInterrupt: 